# Setup

In [1]:
with open("input/16", "r") as fp:
    data = fp.read()

In [2]:
import numpy as np

In [3]:
def parse_line(line):
    
    name = line[6:8]
    flow = int(line.split("=")[1].split(";")[0])
    valves = line.split("valve")[1].split(", ")
    if len(valves[0]) > 2:
        valves[0] = valves[0][-2:]
    
    return name, flow, valves
    

In [6]:
example = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II"""

In [20]:
puzzle = data[:-1].split("\n")

In [7]:
puzzle = example.split("\n")

In [21]:
puzzle_processed = list(map(parse_line, puzzle))

dic_path = {}
dic_power = {}
for c, v, nnn in puzzle_processed:
    dic_path[c] = nnn
    dic_power[c] = v

dic_index = dict([(c, idx) for idx, c in enumerate(dic_power)])
n = len(dic_index)

S_ref = list(filter(lambda x: x[1] == 0, puzzle_processed))
S_ref = set(map(lambda x: x[0], S_ref))


# Part 1

## V3

Remove states that are similar in terms of valves, but do not prune those with lower score

In [149]:
l_tot = len(dic_power)

In [114]:
dic_states = {}

# Score, valves opened
dic_states["AA"] = [(0, S_ref)] # S_ref because no need to open "0" valves.


for clock in range(30):
    print("=== {} ===".format(clock))
    dic_new = dict([(c, []) for c in dic_power])
    for state, memory in dic_states.items():
        # State where we are
        i = dic_index[state]
        for score, val in memory:
            tot = sum([dic_power[x] for x in val])
            
            if state not in val:
                # Open the valve
                dic_new[state].append((tot + score, val | {state}))
                
            if len(val) != l_tot:
                # move somewhere if you can open something
                for s1 in dic_path[state]:
                    dic_new[s1].append((tot + score, val))
                    
            else:
                # No need to move
                dic_new[state].append((tot + score, val))
            
                
    #####################################
    # Clean dic_new to get something OK #
    #####################################
    
    dic_states = {}
    for state, memory in dic_new.items():
        if len(memory) == 0:
            continue
        
        vals_0 = []
        while len(memory) > 0:
            _, S = memory[0]
            G0 = list(filter(lambda x: len(x[1] & S) == len(x[1] | S), memory))
            vmax = list(map(lambda x: x[0], G0))
            vals_0.append((max(vmax), S))
            
            memory = list(filter(lambda x: len(x[1] & S) != len(x[1] | S), memory))
            
        
        dic_states[state] = vals_0
        
    ##################################################################
    # Can improve by removing states included and with a lower score #
    ##################################################################
    for state, memory in dic_states.items():
        memory = sorted(memory, key=lambda x: -len(x[1]))
        
        mem = []
        while len(memory) > 0:
            s0, S = memory.pop()
            l = len(S)
            # Si le score est inférieur, et qu'il y a moins de valves ouvertes, 
            # enlever de la liste
            memory = list(filter(lambda x: False == ((len(S | x[1]) == l) & (s0 >= x[0])), memory))
            mem.append((s0, S))
            
        dic_states[state] = mem
            
        
    #############################################################################
    # Prune if one state is better than another, even if you add the other gate #
    #############################################################################
    # 1: [AA, BB], s0
    # 2: [AA, CC], s1
    

=== 0 ===
=== 1 ===
=== 2 ===
=== 3 ===
=== 4 ===
=== 5 ===
=== 6 ===
=== 7 ===
=== 8 ===
=== 9 ===
=== 10 ===
=== 11 ===
=== 12 ===
=== 13 ===
=== 14 ===
=== 15 ===
=== 16 ===
=== 17 ===
=== 18 ===
=== 19 ===
=== 20 ===
=== 21 ===
=== 22 ===
=== 23 ===


KeyboardInterrupt: 

In [102]:
for key, vals in dic_states.items():
    print(max(map(lambda x: x[0], vals)))
    

1651
1651
1651
1651
1651
1651
1651
1651
1651
1651


## V4: Compress the graph:

- Keep only state where flow OK
- Compute time matrix, to move from `A` to `B`


In [162]:
l_tot = len(dic_index)

M = np.zeros((l_tot, l_tot))

for idx, state in enumerate(dic_index):
    dic_timing = dict([(c, 1000) for c in dic_index])
    dic_timing[state] = 0
    lst = [state]
    
    while len(lst) > 0:
        s0 = lst.pop()
        t = dic_timing[s0] + 1
        for s1 in dic_path[s0]:
            if dic_timing[s1] > t:
                dic_timing[s1] = t
                lst.append(s1)
                
    M[idx] = list(dic_timing.values())

In [165]:
S_clean = set(filter(lambda x: dic_power[x] > 0, dic_power))
S_clean.add("AA")
S_clean = sorted(S_clean)

index = [dic_index[x] for x in S_clean]

M_sub = M[index][:, index].astype(int)
M_sub

dic_index_2 = dict([(c, idx) for idx, c in enumerate(S_clean)])

array([[ 0,  9,  3,  2,  4,  6,  8,  7,  4,  5,  8,  3,  7,  2,  5,  3],
       [ 9,  0, 12,  8,  8,  3,  2,  8, 13, 10,  3,  8, 11, 11,  6,  6],
       [ 3, 12,  0,  5,  4,  9, 11, 10,  7,  2, 11,  6,  7,  5,  8,  6],
       [ 2,  8,  5,  0,  2,  5,  7,  5,  6,  4,  6,  2,  5,  4,  3,  2],
       [ 4,  8,  4,  2,  0,  5,  7,  7,  8,  2,  8,  2,  3,  6,  5,  2],
       [ 6,  3,  9,  5,  5,  0,  2,  8, 10,  7,  3,  5,  8,  8,  6,  3],
       [ 8,  2, 11,  7,  7,  2,  0, 10, 12,  9,  5,  7, 10, 10,  8,  5],
       [ 7,  8, 10,  5,  7,  8, 10,  0, 11,  9,  5,  7, 10,  9,  2,  7],
       [ 4, 13,  7,  6,  8, 10, 12, 11,  0,  9, 12,  7, 11,  2,  9,  7],
       [ 5, 10,  2,  4,  2,  7,  9,  9,  9,  0, 10,  4,  5,  7,  7,  4],
       [ 8,  3, 11,  6,  8,  3,  5,  5, 12, 10,  0,  8, 11, 10,  3,  6],
       [ 3,  8,  6,  2,  2,  5,  7,  7,  7,  4,  8,  0,  5,  5,  5,  2],
       [ 7, 11,  7,  5,  3,  8, 10, 10, 11,  5, 11,  5,  0,  9,  8,  5],
       [ 2, 11,  5,  4,  6,  8, 10,  9,  2,  7, 10,

#### Loop

In [167]:
# time, score, location, valves opened
lst = [(0, 0, "AA", set())]

for t0 in range(30):
    print("=== {} ===".format(t0))
    # Isolate items to update for this round
    lst1 = list(filter(lambda x: x[0] == t0, lst))
    lst  = list(filter(lambda x: x[0] > t0, lst))
    
    
    
    for state in S_clean:
        print(state, end=", ")
        lstx = list(filter(lambda x: x[2] == state, lst1))
        lstx = sorted(lstx, key=lambda x: -len(x[3]))
        
        mem = []
        while len(lstx) > 0:
            _, score, _, gp = lstx.pop()
            l = len(gp)
            
            # Si score plus petit, et que liste incluse, alors supprimer l'élément
            lstx = list(filter(lambda x: False == ((len(gp | x[3]) == l) & (score >= x[1])), lstx))
            mem.append((t0, score, state, gp))
                
            
        lstx = mem
        
        
        
        for _, score, _, gp in lstx:
            weight = sum(map(lambda x: dic_power[x], gp))
            
            if state not in gp:
                # Open the current valve
                lst.append([t0+1, score + weight, state, gp | {state}])
                
            if len(gp) == len(S_clean):
                # No need to move
                lst.append((30, score + weight * (30-t0), gp))
                
            else:
                for s1 in S_clean:
                    i0 = dic_index_2[state]
                    i1 = dic_index_2[s1]
                    dt = M_sub[i0, i1]
                    if dt + t0 <= 30:
                        lst.append([t0 + dt, score + weight*dt, s1, gp])
                
    print()

=== 0 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 1 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 2 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 3 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 4 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 5 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 6 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 7 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 8 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 9 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 10 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 11 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 12 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 13 ===
AA, AG, ES,

KeyboardInterrupt: 

In [159]:
sorted(map(lambda x: x[1], lst), reverse=True)

[1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1651,
 1648,
 1648,
 1648,
 1648,
 1648,
 1648,
 1645,
 1641,
 1641,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1639,
 1638,
 1633,
 1632,
 1632,
 1631,
 1631,
 1629,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1626,
 1624,
 1624,
 1624,
 1624,
 1624,
 1624,
 1624,
 1624,
 1624,
 1624,
 1621,
 1621,
 1618,
 1618,
 1615,
 1614,
 1614,
 1612,
 1612,
 1612,
 1612,
 1612,
 1612,
 1612,
 1612,
 1612,
 1612,
 1612,
 1612,
 1612,
 1612,
 1612,
 1612,

#### V5: If you go on a specific state, then open the door

You can track record of the best score so far, and check if a state can reach it with the best power possible.



In [169]:
SS_clean = set(S_clean)
# time, score, location, valves opened
lst = [(0, 0, "AA", set())]

for t0 in range(30):
    print("=== {} ===".format(t0))
    # Isolate items to update for this round
    lst1 = list(filter(lambda x: x[0] == t0, lst))
    lst  = list(filter(lambda x: x[0] > t0, lst))
    
    
    
    for state in S_clean:
        print(state, end=", ")
        lstx = list(filter(lambda x: x[2] == state, lst1))
        lstx = sorted(lstx, key=lambda x: -len(x[3]))
        
        mem = []
        while len(lstx) > 0:
            _, score, _, gp = lstx.pop()
            l = len(gp)
            
            # Si score plus petit, et que liste incluse, alors supprimer l'élément
            lstx = list(filter(lambda x: False == ((len(gp | x[3]) == l) & (score >= x[1])), lstx))
            mem.append((t0, score, state, gp))
                
            
        lstx = mem
        
        
        
        for _, score, _, gp in lstx:
            weight = sum(map(lambda x: dic_power[x], gp))
            
            if len(gp) == len(S_clean):
                # No need to move
                # Wait until the end
                lst.append((30, score + weight * (30-t0), gp))
                
            else:
                # Visit only un-opened states
                for s1 in SS_clean - gp:
                    i0 = dic_index_2[state]
                    i1 = dic_index_2[s1]
                    dt = M_sub[i0, i1] + 1
                    if dt + t0 <= 30:
                        lst.append([t0 + dt, score + weight*dt, s1, gp | {s1}])
                
    print()

=== 0 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 1 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 2 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 3 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 4 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 5 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 6 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 7 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 8 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 9 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 10 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 11 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 12 ===
AA, AG, ES, MO, OC, OR, QJ, QS, RT, SX, TX, UH, VZ, XT, YB, ZI, 
=== 13 ===
AA, AG, ES,

In [171]:
max(list(map(lambda x: x[1], lst)))

1820

# Part 2: Two location at the same time

Two moves at the same time also

In [22]:
l_tot = len(dic_index)

M = np.zeros((l_tot, l_tot))

for idx, state in enumerate(dic_index):
    dic_timing = dict([(c, 1000) for c in dic_index])
    dic_timing[state] = 0
    lst = [state]
    
    while len(lst) > 0:
        s0 = lst.pop()
        t = dic_timing[s0] + 1
        for s1 in dic_path[s0]:
            if dic_timing[s1] > t:
                dic_timing[s1] = t
                lst.append(s1)
                
    M[idx] = list(dic_timing.values())

In [23]:
S_clean = set(filter(lambda x: dic_power[x] > 0, dic_power))
S_clean.add("AA")
S_clean = sorted(S_clean)

index = [dic_index[x] for x in S_clean]

M_sub = M[index][:, index].astype(int)
M_sub

dic_index_2 = dict([(c, idx) for idx, c in enumerate(S_clean)])

In [ ]:
1645 too low

## One step by one: OK but slow

In [10]:
Tmax = 0
Pmax = sum(dic_power.values()) # If go fast

S_good = set(filter(lambda x: dic_power[x] > 0, dic_power))
S_bad = set(filter(lambda x: dic_power[x] == 0, dic_power))


lst = [("AA", "AA", 0, S_bad)]
for t0 in range(26):
    
    print("=== {} ===".format(t0))
    print("\t Tmax:", Tmax)
    print("\t LST   \t", len(lst))
    
    lst_new = []
    
    # For fast access, format as a dictionary
    dico = {}
    for s0, s1, score, gp in lst:
        if s0 not in dico:
            dico[s0] = {}
            
        if s1 not in dico[s0]:
            dico[s0][s1] = []
        
        dico[s0][s1].append([score, gp])
        
        
    for s0, dic in dico.items():
        for s1, vals in dic.items():
            # Sort by decreasing score
            L1 = sorted(vals, key=lambda x: -x[0])
            
            while len(L1) != 0:
                score, gp = L1.pop(0)
                l = len(gp)
                L1 = list(filter(lambda x: False == ((len(gp | x[1]) == l) & (x[0] <= score)), L1))
                lst_new.append((s0, s1, score, gp))
    
    lst = lst_new
    lst_new = []
            
        
    
    ##############
    for s0, s1, score, gp in lst:
        weight = sum([dic_power[x] for x in gp])
        
        tt = score + weight * (26 - t0)
        if tt > Tmax:
            Tmax = tt
            
        if weight == Pmax:
            # No need to improve
            continue
    
        # No need to process the input if global result too low
        if score + Pmax * (26 - t0) < Tmax:
            continue
        
        
        # Single move
        if s1 not in gp:
            for sx0 in dic_path[s0]:
                lst_new.append((min(sx0, s1), max(sx0, s1), score + weight, gp | {s1}))
        
        if s0 not in gp:
            for sx1 in dic_path[s1]:
                lst_new.append((min(s0, sx1), max(s0, sx1), score + weight, gp | {s0}))
            
            # No move
            if s1 not in gp:
                lst_new.append((min(s0, s1), max(s0, s1), score + weight, gp | {s0, s1}))
                
        
        # Both move
        for sx0 in dic_path[s0]:
            for sx1 in dic_path[s1]:
                lst_new.append((min(sx0, sx1), max(sx0, sx1), score + weight, gp))
    
    lst = lst_new
    

=== 0 ===
	 Tmax: 0
	 LST   	 1
=== 1 ===
	 Tmax: 0
	 LST   	 9
=== 2 ===
	 Tmax: 0
	 LST   	 55
=== 3 ===
	 Tmax: 792
	 LST   	 150
=== 4 ===
	 Tmax: 963
	 LST   	 405
=== 5 ===
	 Tmax: 1029
	 LST   	 738
=== 6 ===
	 Tmax: 1236
	 LST   	 1266
=== 7 ===
	 Tmax: 1340
	 LST   	 1395
=== 8 ===
	 Tmax: 1630
	 LST   	 1661
=== 9 ===
	 Tmax: 1674
	 LST   	 784
=== 10 ===
	 Tmax: 1706
	 LST   	 250
=== 11 ===
	 Tmax: 1706
	 LST   	 102
=== 12 ===
	 Tmax: 1707
	 LST   	 48
=== 13 ===
	 Tmax: 1707
	 LST   	 0
=== 14 ===
	 Tmax: 1707
	 LST   	 0
=== 15 ===
	 Tmax: 1707
	 LST   	 0
=== 16 ===
	 Tmax: 1707
	 LST   	 0
=== 17 ===
	 Tmax: 1707
	 LST   	 0
=== 18 ===
	 Tmax: 1707
	 LST   	 0
=== 19 ===
	 Tmax: 1707
	 LST   	 0
=== 20 ===
	 Tmax: 1707
	 LST   	 0
=== 21 ===
	 Tmax: 1707
	 LST   	 0
=== 22 ===
	 Tmax: 1707
	 LST   	 0
=== 23 ===
	 Tmax: 1707
	 LST   	 0
=== 24 ===
	 Tmax: 1707
	 LST   	 0
=== 25 ===
	 Tmax: 1707
	 LST   	 0


## V5

In [24]:
puzzle_processed = list(map(parse_line, puzzle))

dic_path = {}
dic_power = {}
for c, v, nnn in puzzle_processed:
    dic_path[c] = nnn
    dic_power[c] = v

dic_index = dict([(c, idx) for idx, c in enumerate(dic_power)])
n = len(dic_index)

S_ref = list(filter(lambda x: x[1] == 0, puzzle_processed))
S_ref = set(map(lambda x: x[0], S_ref))


In [25]:
l_tot = len(dic_index)

M = np.zeros((l_tot, l_tot))

for idx, state in enumerate(dic_index):
    dic_timing = dict([(c, 1000) for c in dic_index])
    dic_timing[state] = 0
    lst = [state]
    
    while len(lst) > 0:
        s0 = lst.pop()
        t = dic_timing[s0] + 1
        for s1 in dic_path[s0]:
            if dic_timing[s1] > t:
                dic_timing[s1] = t
                lst.append(s1)
                
    M[idx] = list(dic_timing.values())

In [26]:
S_clean = set(filter(lambda x: dic_power[x] > 0, dic_power))
S_clean.add("AA")
S_clean = sorted(S_clean)

index = [dic_index[x] for x in S_clean]

M_sub = M[index][:, index].astype(int)
M_sub

dic_index_2 = dict([(c, idx) for idx, c in enumerate(S_clean)])

In [27]:
SS_clean = set(S_clean)
S_good = SS_clean

# time0, time1, loc0, loc1, score valves opened
lst = [(0, 0, "AA", "AA", 0, {"AA"})]

Tmax = 0
Pmax = sum(dic_power.values()) # If go fast

T_TOT = 26

for tx in range(T_TOT):
    print("=== {} ===".format(tx))
    # Isolate items to update for this round
    lst1 = list(filter(lambda x: (x[0] == tx) | (x[1] == tx), lst))
    lst  = list(filter(lambda x: False == ((x[0] == tx) | (x[1] == tx)), lst))
    

    print("\t {} ==> {}".format(len(lst), len(lst1)))
    print("TMAX: \t{}".format(Tmax))
    
    lst0 = []
    # Re order
    for t0, t1, s0, s1, score, gp in lst1:
        if t1 < t0:
            lst0.append((t1, t0, s1, s0, score, gp))
            
        else:
            lst0.append((t0, t1, s0, s1, score, gp))
            
    lst1 = lst0
    
    lst1 = []
    # Remove duplicates
    S_time = set(map(lambda x: x[1], lst0))
    
    # Filter by time
    for tt1 in S_time:
        L1 = list(filter(lambda x: x[1] == tt1, lst0))
        
        # Filter by state
        S_states = set([(x[2]+x[3]) for x in L1])
        for state in S_states:
            L2 = list(filter(lambda x: (x[2] == state[:2]) & (x[3] == state[2:]) , L1))
            
            # Sorted by decreasing score
            L2 = sorted(L2, key=lambda x: -x[4])
        
            while len(L2) > 0:
                t0, t1, s0, s1, score, gp = L2.pop(0)
                lg = len(gp)
                L2 = list(filter(lambda x:  False == ((len(gp | x[5]) == lg) & (x[4] <= score)), L2))
                lst1.append((t0, t1, s0, s1, score, gp))
    
    
    
    for t0, t1, s0, s1, score, gp in lst1:
        Tmax = max(score, Tmax)
        XX = sum([dic_power[i] for i in S_good - gp])
        
        if score + XX * (T_TOT - tx) <= Tmax:
            # No need to consider if low score
            continue
        
        if XX == 0:
            # No need for update
            continue
        
        i0 = dic_index_2[s0]
        i1 = dic_index_2[s1]
            
        if (tx == t0) & (tx == t1):
            # Do transition for both
            
            for s0x in S_good - gp:
                i0x = dic_index_2[s0x]
                dt0 = M_sub[i0, i0x] + 1
                scnew = score + max(0, T_TOT - tx - dt0) * dic_power[s0x]
                
                for s1x in S_good - gp - {s0x}:
                    i1x = dic_index_2[s1x]
                    dt1 = M_sub[i1, i1x] + 1
                    scnew1 = scnew +  max(0, T_TOT - tx - dt1) * dic_power[s1x]
                    lst.append((t0+dt0, t1+dt1, s0x, s1x, scnew1, gp | {s0x, s1x}))
            
        
            
        elif t0 == tx: # Only t0 == tx
            # Do transition for once
            for s0x in S_good - gp:
                i0x = dic_index_2[s0x]
                dt0 = M_sub[i0, i0x] + 1
                scnew = score + max(0, T_TOT - t0 - dt0) * dic_power[s0x]
                lst.append((t0 + dt0, t1, s0x, s1, scnew, gp | {s0x}))
            
        else:
            for s1x in S_good - gp:
                i1x = dic_index_2[s1x]
                dt1 = M_sub[i1, i1x] + 1
                scnew = score + max(0, T_TOT - t1 - dt1) * dic_power[s1x]
                lst.append((t0, t1 + dt1, s0, s1x, scnew, gp | {s1x}))
            


=== 0 ===
	 0 ==> 1
TMAX: 	0
=== 1 ===
	 210 ==> 0
TMAX: 	0
=== 2 ===
	 210 ==> 0
TMAX: 	0
=== 3 ===
	 156 ==> 54
TMAX: 	0
=== 4 ===
	 662 ==> 144
TMAX: 	855
=== 5 ===
	 2760 ==> 164
TMAX: 	1208
=== 6 ===
	 4326 ==> 514
TMAX: 	1208
=== 7 ===
	 10293 ==> 1167
TMAX: 	1559
=== 8 ===
	 22048 ==> 2047
TMAX: 	1763
=== 9 ===
	 41633 ==> 5306
TMAX: 	1842
=== 10 ===
	 92764 ==> 11234
TMAX: 	1842
=== 11 ===
	 195163 ==> 17953
TMAX: 	2115
=== 12 ===
	 333569 ==> 36100
TMAX: 	2115
=== 13 ===
	 591053 ==> 66840
TMAX: 	2115
=== 14 ===
	 994613 ==> 122730
TMAX: 	2202
=== 15 ===
	 1688614 ==> 200961
TMAX: 	2305
=== 16 ===
	 2636341 ==> 310686
TMAX: 	2305
=== 17 ===
	 3529201 ==> 519414
TMAX: 	2417
=== 18 ===
	 3922483 ==> 701945
TMAX: 	2446
=== 19 ===
	 3621827 ==> 814995
TMAX: 	2446
=== 20 ===
	 2984888 ==> 819274
TMAX: 	2495
=== 21 ===
	 2253233 ==> 756043
TMAX: 	2542
=== 22 ===
	 1579792 ==> 681773
TMAX: 	2542
=== 23 ===
	 1033357 ==> 548874
TMAX: 	2564
=== 24 ===
	 622754 ==> 410935
TMAX: 	2602
==

In [28]:
Tmax

2602

In [ ]:
1707 # Normal

In [261]:
Tmax

2378